In [1]:
#import necessary libraries
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import os
import tecplot_io as tec
import matplotlib.ticker as tck
from scipy.integrate import solve_ivp
from scipy.io import loadmat
import pysindy as ps

In [2]:
#parameters
#Declare some variable 
nturbinex = 4
nturbiney = 4
nturbine = 16
#Diameter of turbine
D = 80
#Other variable
Sx = 7
Sy = 7
#kappa = 0.4
#nu = 1.511e-5
#PEX = 1.45444104333
#PEY = 8.72664625997
#hbar = 0.46
#uinfty = 2.54390548295
dt = 0.68543297937
#Rotational angular period
T_turb = 42.84
U_star = 0.356
H_hub = 70
NPX=192
NPY=192
NPZ=65
#Mean finite velocity
U = 11.5258407161
tis = 10000
tie = 15000
tii = 100
nti = int((tie - tis) / tii + 1)

In [3]:
#change working directory
path  = 'd:\post\LASD'
os.chdir(path)
print("current working directory is: {0}".format(os.getcwd()))

current working directory is: d:\post\LASD


In [4]:
def read_data(path, start_id, end_id, step_id):
    """
    Read data from tecplot file
    """
    os.chdir(path)
    nti = int((end_id - start_id) / step_id + 1)
    X_data = np.zeros((nti, NPY*NPX))
    for it in range(0, nti, 1):
        ti = tis + tii * it
        fname = 'POST_U_2D2_{:010d}_0002.dat'.format(ti)
        f = tec.tecplot_reader(fname, [NPY, NPX, 6], 2)
        #reshape data into one dimension
        u = f[:, :, 3].reshape(NPY * NPX)
        X_data[it, :] = u
    return X_data

In [5]:
def get_coordinate(path):
    """
    Read coordinate from tecplot file
    """
    os.chdir(path)
    fname = 'POST_U_2D2_{:010d}_0002.dat'.format(tis)
    f = tec.tecplot_reader(fname, [NPY, NPX, 6], 2)
    x = f[:, :, 0].reshape(NPY * NPX)
    y = f[:, :, 1].reshape(NPY * NPX)
    z = f[:, :, 2].reshape(NPY * NPX)
    return x, y, z

In [6]:
def store_data(path):
    """
    Read data from tecplot file
    """
    os.chdir(path)
    
    
    for it in range(0, nti, 1):
        ti = tis + tii * it
        fname = 'POST_U_2D2_{:010d}_0002.dat'.format(ti)
        f = tec.tecplot_reader(fname, [NPY, NPX, 6], 2)
        #reshape data into one dimension
        u = f[:, :, 3].reshape(NPY * NPX)
        X_data[it, :] = u
    return X_data

In [7]:
#test function
from scipy import linalg
X_data = read_data(path + '\POST_U_2D2_0002', tis, tie, tii)
X = X_data[:-1, :].T # First time step
Y = X_data[1:, :].T # Time-shifted matrix so that Y = AX
U,S,V = linalg.svd(X)
print(U.shape, S.shape, V.shape)


In [ ]:
#compute DMD
r = 21  #truncate at 21 modes
U_r = U[:,1:r]
S_r = S[1:r, 1:r]
V_r = V[:,1:r]
Atilde = U_r.dot(Y).dot(V_r.T)/S_r